In [1]:
import pandas as pd

In [40]:
test1 = pd.read_csv('한국서부발전(주)_신재생에너지발전량_20220228.csv', encoding='cp949')
test2 = pd.read_csv('한국서부발전(주)_태양광 발전 현황_20220731.csv', encoding='cp949')

In [50]:
test2.head()

,발전기명,년월일,설비용량(MW),01시,02시,03시,04시,05시,06시,07시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,세종시폐기물매립장태양광,2017-01-01,1.628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,742080.0,334440.0,87240.0,2880.0,4320.0,0.0,0.0,0.0,0.0,0.0
1,세종시폐기물매립장태양광,2017-01-02,1.628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,606360.0,501120.0,158280.0,2760.0,3360.0,0.0,0.0,0.0,0.0,0.0
2,세종시폐기물매립장태양광,2017-01-03,1.628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,881040.0,557040.0,128640.0,2280.0,3000.0,0.0,0.0,0.0,0.0,0.0
3,세종시폐기물매립장태양광,2017-01-04,1.628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,677280.0,502320.0,112080.0,1800.0,2760.0,0.0,0.0,0.0,0.0,0.0
4,세종시폐기물매립장태양광,2017-01-05,1.628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,259560.0,138840.0,45000.0,2520.0,2880.0,0.0,0.0,0.0,0.0,0.0


In [44]:
df_original = pd.read_csv('한국서부발전(주)_신재생에너지발전량_20220228.csv', encoding='cp949')
# Adjusting for the 24시 issue during the combination of date and time
hours = [f'{i:02d}시' for i in range(1, 25)]
adjusted_dates = []
adjusted_powers = []
adjusted_years = []
adjusted_addresses = []
adjusted_capacities = []
adjusted_name = []

for index, row in df_original.iterrows():
    for hour in hours:
        if hour == "24시":
            adjusted_date = pd.Timestamp(row['날짜']) + pd.Timedelta(days=1)
            adjusted_hour = "00시"
        else:
            adjusted_date = pd.Timestamp(row['날짜'])
            adjusted_hour = hour
        
        power = row[hour]
        adjusted_dates.append(adjusted_date + pd.Timedelta(hours=int(adjusted_hour.replace('시', ''))))
        adjusted_powers.append(power)
        adjusted_years.append(adjusted_date.year)
        adjusted_name.append(row['발전기명'])
        adjusted_addresses.append(row['주소지'])
        adjusted_capacities.append(row['용량_메가와트'])

# Constructing the final dataframe with additional columns
df_transformed_with_extra = pd.DataFrame({
    'year': adjusted_years,
    'power': adjusted_powers,
    'date': adjusted_dates,
    'name' : adjusted_name,
    'address': adjusted_addresses,
    'capacities': adjusted_capacities
})
df_filtered = df_transformed_with_extra[df_transformed_with_extra['address'] != '전라남도 영암군 삼호읍 에프원로 2 영암 F1 태양광 발전소']
df_filtered = df_filtered.drop(columns = 'year')
df_filtered = df_filtered[df_filtered['power'] != 0.0]
df_filtered['land'] = df_filtered['capacities']*13200
df_filtered['power/land'] = df_filtered['power']/df_filtered['land']
df_filtered = df_filtered.drop(columns = 'address')

In [60]:
df_filtered.tail(20)

,power,date,name,capacities,land,power/land
1519431,40891.0,2017-01-03 16:00:00,태안태양광발전기,0.12,1584.0,25.815025
1519432,12470.0,2017-01-03 17:00:00,태안태양광발전기,0.12,1584.0,7.872475
1519449,3845.0,2017-01-02 10:00:00,태안태양광발전기,0.12,1584.0,2.427399
1519450,11616.0,2017-01-02 11:00:00,태안태양광발전기,0.12,1584.0,7.333333
1519451,34248.0,2017-01-02 12:00:00,태안태양광발전기,0.12,1584.0,21.621212
1519452,31282.0,2017-01-02 13:00:00,태안태양광발전기,0.12,1584.0,19.748737
1519453,52474.0,2017-01-02 14:00:00,태안태양광발전기,0.12,1584.0,33.127525
1519454,35947.0,2017-01-02 15:00:00,태안태양광발전기,0.12,1584.0,22.693813
1519455,19061.0,2017-01-02 16:00:00,태안태양광발전기,0.12,1584.0,12.033460
1519456,4709.0,2017-01-02 17:00:00,태안태양광발전기,0.12,1584.0,2.972854


In [62]:
# Assuming 'date' column is in string format, we first convert it to datetime for accurate min and max operations
df_filtered['date'] = pd.to_datetime(df_filtered['date'])

# Group by 'name' and calculate min and max 'date'
date_range_by_name = df_filtered.groupby('name')['date'].agg(['min', 'max']).reset_index()

date_range_by_name

,name,min,max
0,[서인천]발전부지내 태양광 1단계,2018-01-01 09:00:00,2022-02-28 20:00:00
1,[서인천]발전부지내 태양광 1단계ESS,2019-05-24 01:00:00,2022-03-01 00:00:00
2,[서인천]발전부지내 태양광 2단계,2018-01-01 09:00:00,2022-02-28 18:00:00
3,[평택]발전부지내 1단계 태양광 ESS,2019-05-24 01:00:00,2022-03-01 00:00:00
4,[평택]발전부지내 태양광 1단계,2018-01-02 10:00:00,2022-02-28 21:00:00
5,군산복합2단계태양광,2018-01-01 09:00:00,2022-02-28 22:00:00
6,군산복합2단계태양광ESS,2019-05-25 01:00:00,2022-03-01 00:00:00
7,대전 학하 연료전지,2021-11-11 14:00:00,2022-03-01 00:00:00
8,삼랑진태양광#1,2018-01-01 09:00:00,2022-02-28 19:00:00
9,삼랑진태양광#2,2018-01-01 08:00:00,2022-02-28 18:00:00


In [63]:
date_range_by_name.to_csv("date_range_by_name.csv", encoding = 'cp949')

In [49]:
# 발전기명에 대한 고유값(종류)를 얻기
unique_generators = df_filtered['address'].unique()

# 각 발전기명별 행의 수를 얻기
generator_counts = df_filtered['address'].value_counts()

#print("발전기 종류:", unique_generators)
print("\n각 발전기별 행의 수:\n", generator_counts)


각 발전기별 행의 수:
 address
충남 태안군 원북면 발전로 447-40 태안발전본부     226141
인천광역시 서구 장도로 57                   93179
경기도 평택시 포승읍 남양만로 175-2 평택발전본부     48992
충남 연기군 금남면 두만리 68-8외 39필지         44258
충남 연기군 남면 고정리 599외 16필지           42859
전북 군산시 구암 3.1로 91-5 군산발전처         34682
경상남도 밀양시 삼랑진읍 행곡로 26-27           29449
충남 연기군 남면 송원리 530외 12필지           29400
아산시 영인면 구성리 282-3 일원              25063
전남 화순군 동면 청궁리 산1번지                24594
전남 고흥군 도화면 봉산리 466-7~10           20860
경기도 시흥시 광석동 146-1                 18923
경기도 평택시 포승읍 석정리 278-1             15303
경기도 안산시 단원구 와동884                 14012
경기도 안산시 단원구 신길동 산11-12            13926
경기도 안산시 단원구 목내동 471-1             13917
경기도 안산시 단원구 성곡동639-1              13727
경기도 안산시 단원구 초지동636-1              13520
천안시 동남구 청당동 613 청수에너지             13258
경기도 화성시 남양읍 북양리 산51                9629
경기도 의왕시 이동 714번지 일원                6289
전남 장흥군 유치면 용문리 산4번지 외 3필지          3855
대전광역시 유성구 복용동 산22-55번지             2626
Name: count, dtype: int64


In [48]:
# 발전기명에 대한 고유값(종류)를 얻기
unique_generators = df_filtered['name'].unique()

# 각 발전기명별 행의 수를 얻기
generator_counts = df_filtered['name'].value_counts()

#print("발전기 종류:", unique_generators)
print("\n각 발전기별 행의 수:\n", generator_counts)


각 발전기별 행의 수:
 name
세종시폐기물매립장태양광ESS          24296
태안태양광2단계_ESS             23942
서인천연료전지                  23318
서인천연료전지2                 21208
화순풍력                     21012
[서인천]발전부지내 태양광 1단계ESS    18964
군산복합2단계태양광ESS            18960
안산연성정수장태양광               18923
[평택]발전부지내 1단계 태양광 ESS    18916
태안_ESS#2_2단계_#1옥내저탄장     18901
세종시폐기물매립장태양광             18563
태안_ESS#2_2단계_건축물         18406
태안_ESS#2_#1수상            18404
태안_ESS#2_1단계_건축물         18346
태안태양광발전기                 17165
태안부지 태양광 2단계(대지)         16204
수질복원센터태양광                15902
태안소수력#1                  15852
군산복합2단계태양광               15722
태안9·10호기태양광              15678
[평택]발전부지내 태양광 1단계        15637
[서인천]발전부지내 태양광 1단계       15446
새빛태양광                    15392
석정태양광발전소                 15303
태안발전부지태양광1호기             15074
삼랑진태양광#2                 14970
태안부지 태양광 2단계(건축물)        14865
세종시자전거도로태양광#3            14800
세종시자전거도로태양광#2            14732
세종시자전거도로태양광              14726
삼랑진태양광#1                 14479
평택2복합태양광           

In [54]:
# Selecting distinct 'name' and their corresponding 'capacity' values
distinct_name_capacity = df_filtered[['name', 'capacities']].drop_duplicates()

distinct_name_capacity

,name,capacities
85807,[서인천]발전부지내 태양광 1단계,1.099
87216,[서인천]발전부지내 태양광 1단계ESS,1.000
90055,[서인천]발전부지내 태양광 2단계,0.090
91480,[평택]발전부지내 1단계 태양광 ESS,2.000
94303,[평택]발전부지내 태양광 1단계,2.220
95719,군산복합2단계태양광,0.455
97145,군산복합2단계태양광ESS,0.500
99960,대전 학하 연료전지,6.160
101383,삼랑진태양광#1,2.000
102799,삼랑진태양광#2,1.000


In [ ]:
세종시폐기물매립장태양광ESS          24296
태안태양광2단계_ESS             23942
서인천연료전지                  23318
서인천연료전지2                 21208
화순풍력                     21012
[서인천]발전부지내 태양광 1단계ESS    18964
군산복합2단계태양광ESS            18960
안산연성정수장태양광               18923
[평택]발전부지내 1단계 태양광 ESS    18916
태안_ESS#2_2단계_#1옥내저탄장     18901
세종시폐기물매립장태양광             18563
태안_ESS#2_2단계_건축물         18406
태안_ESS#2_#1수상            18404
태안_ESS#2_1단계_건축물         18346
태안태양광발전기                 17165
태안부지 태양광 2단계(대지)         16204
수질복원센터태양광                15902
태안소수력#1                  15852
군산복합2단계태양광               15722
태안9·10호기태양광              15678
[평택]발전부지내 태양광 1단계        15637
[서인천]발전부지내 태양광 1단계       15446
새빛태양광                    15392
석정태양광발전소                 15303
태안발전부지태양광1호기             15074
삼랑진태양광#2                 14970
태안부지 태양광 2단계(건축물)        14865
세종시자전거도로태양광#3            14800
세종시자전거도로태양광#2            14732
세종시자전거도로태양광              14726
삼랑진태양광#1                 14479
평택2복합태양광                 14439
[서인천]발전부지내 태양광 2단계       14243
안산와동태양광                  14012
안산공단태양광                  13926
안산목내태양광                  13917
안산성곡태양광                  13727
태안저탄장태양광1호기              13526
안산초지태양광                  13520
한국서부발전(주)태양광발전소 ESS      13504
세종시수질복원센터태양광연계 ESS       13498
아산시 폐기물 매립장 태양광연계 ESS    13363
천안청수연료전지                 13258
아산시 폐기물 매립장 태양광          11700
화성 남양 연료전지                9629
한국서부발전(주)태양광발전소           7356
의왕 연료전지                   6289
태안소수력#2                   4386
장흥풍력                      3855
화순풍력 ESS                  3582
대전 학하 연료전지                2626